In [3]:
# ADMM adaptive lasso using ProximalOperators with line and golden section search
# Automatic tuning of learning rate and regularization parameter
# This code only only changes the ratio between test data and train data, and changes the input file
# The original source code from https://github.com/patwa67/AUTALASSO

using ProximalOperators
using DelimitedFiles
using Statistics
using LinearAlgebra

In [4]:
# Function for Golden section search to optimize lambda
function gss_opt(alam, blam, tolgss, Xtesthot, ytest,abscovinv,maxnorm)
  lama =alam*maxnorm # Lower lambda
  lamb =blam*maxnorm # Higher lambda
  gr = (sqrt(5.0) + 1.0) / 2.0 # Golden section ratio
  toladmm = 1e-4 # Convergence tolerance for ADMM
  fc = lasso_admm(Xtrainhot, ytrain, lama, zero(Xtrainhot[1,:]),zero(Xtrainhot[1,:]),f, abscovinv,toladmm)
  lossc= 0.5*norm(Xtesthot*fc[1].-ytest)^2 # Test error for initial lower lambda
  fd = lasso_admm(Xtrainhot, ytrain, lamb, zero(Xtrainhot[1,:]),zero(Xtrainhot[1,:]),f, abscovinv,toladmm)
  lossd= 0.5*norm(Xtesthot*fd[1].-ytest)^2 # Test error for initial higher lambda
  iter = 2
  meanlam = zero(1.0:100.0)
  #meanlam[iter] = (lama+lamb)/2
  meanloss = zero(1.0:100.0)
  #meanloss[1] = max(lossc,lossd)
  #meanloss[iter] = (lossc+lossd)/2
  lamc = lamb - (lamb - lama) / gr
  lamd = lama + (lamb - lama) / gr
  println("lossc =$lossc")
  println("lossd =$lossd")
  println("lambdaa =$lama")
  println("lambdab =$lamb")
  println("lambdac =$lamc")
  println("lambdad =$lamd")
  iter = 2
  nodrun = 0
  while abs(lamc - lamd)/((lamc + lamd)/2.0) > tolgss # Run GSS until convergence
    iter = iter+1
    if iter == 3
    fc = lasso_admm(Xtrainhot, ytrain, lamc, fc[1],fc[2],f, abscovinv,toladmm)
    lossc= 0.5*norm(Xtesthot*fc[1].-ytest)^2 # Test error for initial lower lambda
    fd = lasso_admm(Xtrainhot, ytrain, lamd, fd[1],fd[2],f, abscovinv,toladmm)
    lossd= 0.5*norm(Xtesthot*fd[1].-ytest)^2 # Test error for initial higher lambda
    else
    if nodrun==1
    fc = lasso_admm(Xtrainhot, ytrain, lamc, fc[1],fc[2],f, abscovinv,toladmm)
    lossc= 0.5*norm(Xtesthot*fc[1].-ytest)^2 # Test error for initial lower lambda
    else
    fd = lasso_admm(Xtrainhot, ytrain, lamd, fd[1],fd[2],f, abscovinv,toladmm)
    lossd= 0.5*norm(Xtesthot*fd[1].-ytest)^2 # Test error for initial higher lambda
    end
    end
    meanlam[iter] = (lamc+lamd)/2.0
    meanloss[iter] = (lossc+lossd)/2.0
    # Stop GSS if test MSE is increased two consecutive iterations
    if (meanloss[iter] > meanloss[iter-1])&&(meanloss[iter-1] > meanloss[iter-2])
      break
    end
    if lossc < lossd
      lamb = lamd
      fd=fc
      lossd=lossc
      nodrun=1
    else
      lama = lamc
      fc=fd
      lossc=lossd
      nodrun=0
    end
    lamc = lamb - (lamb - lama) / gr
    lamd = lama + (lamb - lama) / gr
    #println("lossc =$lossc")
    #println("lossd =$lossd")
    println("lambdac =$lamc")
    println("lambdad =$lamd")
  end
  # Final ADMM for optimized lambda
  lamopt = meanlam[iter-2]
  fmean1 = (fc[1]+fd[1])/2.0
  fmean2 = (fc[2]+fd[2])/2.0
  fopt = lasso_admm(Xtrainhot, ytrain, lamopt, fmean1,fmean2,f, abscovinv,toladmm)
  lossopt= 0.5*norm(Xtesthot*fopt[1].-ytest)^2
  acc = cor(Xtesthot*fopt[1],ytest)

  return lamopt,fopt,lossopt,acc
end

gss_opt (generic function with 1 method)

In [5]:
# Function for Proximal ADMM lasso with line search
function lasso_admm(Xtrainhot, ytrain, lam, theta, beta, f, abscovinv,tol; maxit=5000)
  u = zero(theta)
  grad = zero(theta)
  lamw = lam*abscovinv # Regularization parameter times weights
  g = NormL1(lamw) # Regularization function
  c = 0.5
  lr = 1.0
  loss(theta) = 0.5*norm(Xtrainhot*theta-ytrain)^2 # Loss function for line search
  for it = 1:maxit
    # Line search
    it % 8 == 1 && (grad = Xtrainhot'*(Xtrainhot*beta-ytrain))
    while  it % 20 == 2 && loss(theta) > (loss(beta) + grad'*(-beta) + (1.0/(2.0*lr))*norm(-beta)^2)
      lr = lr * c
      #println(lr)
    end
    gam = lr
    # ADMM perform f-update step
    prox!(beta, f, theta - u, gam)
    # ADMM perform g-update step
    prox!(theta, g, beta + u, gam)
    # Stopping criterion for ADMMM
    if norm(beta-theta, Inf) <= tol*(1+norm(u, Inf))
      break
    end
    # Dual update
    u .+= beta - theta
    #print(it)
  end
  return theta,beta,tol
end

lasso_admm (generic function with 1 method)

In [6]:
data_file_path = "/Users/dong/GWAS/AtPolyDB/Germ_22.csv"

"/Users/dong/GWAS/AtPolyDB/Germ_22.csv"

In [7]:
# Read Germ_22.csv data, and partition into train and test parts
X = readdlm(data_file_path,',')
ytot = (X[:,1].-mean(X[:,1])) # Center y to mean zero
ytrain = ytot[1:140]
Xtest= X[141:size(X)[1],2:size(X)[2]]
ytest = ytot[141:size(X)[1]]
Xtrain = X[1:140,2:size(X)[2]]

140×214051 Matrix{Float64}:
 0.0  0.0  0.0  2.0  2.0  0.0  0.0  2.0  …  0.0  0.0  0.0  0.0  0.0  0.0  0.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0     0.0  2.0  0.0  0.0  2.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  2.0     0.0  0.0  0.0  0.0  0.0  2.0  0.0
 0.0  2.0  0.0  0.0  0.0  2.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  2.0  0.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0  …  2.0  2.0  0.0  0.0  2.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0     2.0  2.0  0.0  0.0  2.0  0.0  0.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0     2.0  2.0  0.0  0.0  2.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  0.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  2.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  2.0     0.0  2.0  2.0  0.0  2.0  0.0  0.0
 0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0  …  0.0  0.0  0.0  0.0  0.0  0.0  2.0
 0.0  0.0  0.0  0.0  0.0  2.0  0.0  0.0     0.0  0.0  0.0  0.0  0.0  0.0  0.0
 2.0  2.0  2.0  0.0  0.0  0.0  0.0  

In [8]:
# One hot encoding training data
Xtrain0 = copy(Xtrain)
Xtrain1 = copy(Xtrain)
Xtrain2 = copy(Xtrain)
Xtrain0[Xtrain0.==1] .= 2
Xtrain0[Xtrain0.==0] .= 1
Xtrain0[Xtrain0.==2] .= 0
Xtrain1[Xtrain1.==2] .= 0
Xtrain2[Xtrain2.==1] .= 0
Xtrain2[Xtrain2.==2] .= 1
Xtrainhot = hcat(Xtrain0,Xtrain1,Xtrain2)
# Set unimportant allocations to zero
Xtrain0 = 0
Xtrain1 = 0
Xtrain2 = 0
Xtrain = 0

0

In [9]:
# One hot encoding test data
Xtest0 = copy(Xtest)
Xtest1 = copy(Xtest)
Xtest2 = copy(Xtest)
Xtest0[Xtest0.==1] .= 2
Xtest0[Xtest0.==0] .= 1
Xtest0[Xtest0.==2] .= 0
Xtest1[Xtest1.==2] .= 0
Xtest2[Xtest2.==1] .= 0
Xtest2[Xtest2.==2] .= 1
Xtesthot = hcat(Xtest0,Xtest1,Xtest2)
# Set unimportant allocations to zero
Xtest0 = 0
Xtest1 = 0
Xtest2 = 0
Xtest = 0

0

In [10]:
# Factor for initial lower lambda
alam = 0.0001
# Factor for initial upper lambda
blam = 1.0
# Convergence factor for lambda in gss_opt
tolgss = 0.01
# Find lambda where all reg coeff are zero
maxnorm = norm(Xtrainhot'*ytrain, Inf)
# The least squares loss function
f = LeastSquares(Xtrainhot, ytrain)
# Inverse covariances to be used as weights in the adaptive lasso
abscovinv = 1.0./abs.(cov(Xtrainhot,ytrain))

642153×1 Matrix{Float64}:
  79.42857142857142
 555.999999999997
 185.3333333333329
  92.66666666666654
 138.99999999999974
  39.71428571428568
  55.59999999999998
  61.77777777777783
 139.0
  92.66666666666654
  92.66666666666659
 139.0
  79.42857142857136
   ⋮
  46.33333333333341
 277.99999999999926
 111.2
 138.99999999999991
 556.0000000000007
  46.3333333333333
  61.77777777777763
  69.5
 111.2
  42.76923076923073
 185.3333333333341
 556.0000000000015

In [11]:
# Run AUTALASSO with timing
@time res = gss_opt(alam, blam, tolgss, Xtesthot, ytest,abscovinv,maxnorm)


lossc =10.896296891475343
lossd =6.335104854926745
lambdaa =0.0015372881355932223
lambdab =15.372881355932222
lambdac =5.872868269264837
lambdad =9.501550374802978
lambdac =9.501550374802978
lambdad =11.74419925039408
lambdac =11.744199250394082
lambdad =13.130232480341117
lambdac =13.13023248034112
lambdad =13.986848125985185
lambdac =13.986848125985187
lambdad =14.516265710288156
lambdac =14.516265710288156
lambdad =14.843463771629253
lambdac =14.843463771629253
lambdad =15.045683294591125
lambdac =15.045683294591123
lambdad =15.170661832970351
 81.752549 seconds (1.44 M allocations: 8.819 GiB, 2.33% gc time, 1.57% compilation time)


(14.251556918136671, ([0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0  …  0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0], [-5.9267935182808864e-6, 2.775040701640168e-6, -2.0650071581996394e-6, -3.1643218784353964e-6, -1.2003965977980412e-6, 1.0404117252556457e-5, -1.2885101740415486e-5, -6.037383720332042e-6, -3.127032573142194e-6, -2.824891434268157e-6  …  8.797335047563912e-6, 2.8518314048665735e-7, -1.1962537255428639e-6, 1.5120248109351262e-5, 3.791954850346985e-6, 3.189416092945241e-6, -1.981100244043877e-6, 7.969753093508081e-6, 1.6694917351013348e-6, 1.99881810135178e-6], 0.0001), 6.335104854926745, NaN)

In [12]:
output_dir = "/Users/dong/GWAS/Autalasso/" 


"/Users/dong/GWAS/Autalasso/"

In [13]:
# Save regression coefficients, lambda, MSE and ACC to text files
writedlm(joinpath(output_dir, "outbeta_Germ22.txt"), res[2][1])
writedlm(joinpath(output_dir, "outlambda_Germ22.txt"), res[1])
writedlm(joinpath(output_dir, "outMSE_Germ22.txt"), res[3]/length(ytest)*2)
writedlm(joinpath(output_dir, "outACC_Germ22.txt"), res[4])

In [14]:
# Predict observations for test data
ytesthat = Xtesthot*res[2][1]

37-element Vector{Float64}:
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 ⋮
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0
 0.0